# Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import io

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

# Load Dataset

In [2]:
df = pd.read_csv("Reviews.csv",index_col=0)

In [3]:
df.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
Id,,,,,,,,,
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


We will be classifying a review as positive sentiment if it has a score greater than 5. The reviews with score less than 5 are taken as negative sentiment

In [4]:
df["review"] = df["Score"].apply(lambda x: 0 if x<4 else 1)
df = df[["Text","review"]]

In [5]:
df

,Text,review
Id,,
1,I have bought several of the Vitality canned d...,1
2,Product arrived labeled as Jumbo Salted Peanut...,0
3,This is a confection that has been around a fe...,1
4,If you are looking for the secret ingredient i...,0
5,Great taffy at a great price. There was a wid...,1
...,...,...
568450,Great for sesame chicken..this is a good if no...,1
568451,I'm disappointed with the flavor. The chocolat...,0
568452,"These stars are small, so you can give 10-15 o...",1


In [6]:
def text_processing(text):
    text = re.sub(r'\d+', '',text) # remove any numbers
    text = re.sub('<.*?>','',text) # remove any html tags
    text = re.sub(r'https?:\\/\\/.*[\\r\\n]*', '', text, flags=re.MULTILINE) # remove any URLs
    return text

df["Text"] = df["Text"].apply(lambda x: text_processing(x))
df

,Text,review
Id,,
1,I have bought several of the Vitality canned d...,1
2,Product arrived labeled as Jumbo Salted Peanut...,0
3,This is a confection that has been around a fe...,1
4,If you are looking for the secret ingredient i...,0
5,Great taffy at a great price. There was a wid...,1
...,...,...
568450,Great for sesame chicken..this is a good if no...,1
568451,I'm disappointed with the flavor. The chocolat...,0
568452,"These stars are small, so you can give - of th...",1


### Balancing Training data

The dataset has almost 4 times the positive reviews compared to negative reviews. To counter this we can sample only a part of the positive reviews

In [7]:
df["review"].value_counts() # 1 for positive sentiment and 0 for negative sentiment

1    443777
0    124677
Name: review, dtype: int64

In [8]:
positive_reviews = df[df.review == 1]
negative_reviews = df[df.review == 0]

positive_reviews = positive_reviews.sample(n=len(negative_reviews)) # sample positive examples whose number is equal to the negative examples

df = positive_reviews.append(negative_reviews).reset_index(drop=True)

In [9]:
df
# The data is not shuffled right now, but it can be shuffled once we call the train test split function

,Text,review
0,I received this freezer tray along with the Be...,1
1,I have tried many OOlong teas and this is by f...,1
2,"Its a really delicious earthy, herbal tea. Eve...",1
3,I just got the product yesterday. I used it la...,1
4,This was a gift for mom who had run out. She ...,1
...,...,...
249349,I just bought this soup today at my local groc...,0
249350,This soup is mostly broth. Although it has a k...,0
249351,"It is mostly broth, with the advertised / cup ...",0
249352,I had ordered some of these a few months back ...,0


The dataset has 249354 reviews out of which half are positive while the orher half is negative

In [10]:
sentences = df["Text"].values
labels = df["review"].values

The dataset is now split into training and testing sets of 80% and 20% respectively

In [11]:
# Define training and testing sets
train_sentences,test_sentences,train_labels,test_labels = train_test_split(sentences,labels,test_size=0.2,shuffle=True)

# Tokenization

The keras `Tokenizer` class helps in tokenizing the words to form a vocabulary of all the words in the training set.
These tokens are fed into the `Embedding` layer to output the respective embedding vectors

These tokens are then put into sequences based on the sentence. `max_length` is specified to either pad the sentence or truncate it to a specified length.
Any padding or truncation is done from the end of a sentence.
The keyword `'post'` needs to be specified to the pad_sequences method.
Specifying `vocab_size = 10000` tells the tokenizer to only take the most common 10000 words used in the training set.
`max_length` and `vocab_size` are hyperparameters which can be tuned to include more data to be input to the network

In [12]:
embed_dim = 64 # dimension of the embedding layer
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
vocab_size = 10000
max_length = 400

In [13]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok) #Define tokenizer
tokenizer.fit_on_texts(train_sentences) # Assign tokens based on words on training set

train_sequences = tokenizer.texts_to_sequences(train_sentences) # Create sequences based on tokens for the training set

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type) # pad/truncate zeros at the end for a length of 'max_length' 


test_sequences = tokenizer.texts_to_sequences(test_sentences) # similar preprocessing for test set
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Define Model

`tf.keras.layers.Bidirectional` layer helps in the LSTM doing forward pass on both directions of the sequence. This helps for words in the later parts of the sentence affect the activations on earlier words

In [22]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(8,activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(lr=0.0006)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 64)           640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                24832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0

In [23]:
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels),batch_size=128)

Epoch 1/10
1559/1559 [==============================] - 64s 41ms/step - loss: 0.4298 - accuracy: 0.8268 - val_loss: 0.3147 - val_accuracy: 0.8721
Epoch 2/10
1559/1559 [==============================] - 62s 40ms/step - loss: 0.3356 - accuracy: 0.8789 - val_loss: 0.2782 - val_accuracy: 0.8898
Epoch 3/10
1559/1559 [==============================] - 62s 40ms/step - loss: 0.2912 - accuracy: 0.8968 - val_loss: 0.2714 - val_accuracy: 0.8895
Epoch 4/10
1559/1559 [==============================] - 62s 40ms/step - loss: 0.2604 - accuracy: 0.9093 - val_loss: 0.2528 - val_accuracy: 0.8994
Epoch 5/10
1559/1559 [==============================] - 62s 40ms/step - loss: 0.2336 - accuracy: 0.9192 - val_loss: 0.2557 - val_accuracy: 0.9031
Epoch 6/10
1559/1559 [==============================] - 62s 40ms/step - loss: 0.2183 - accuracy: 0.9245 - val_loss: 0.2496 - val_accuracy: 0.9056
Epoch 7/10
1559/1559 [==============================] - 62s 40ms/step - loss: 0.1996 - accuracy: 0.9321 - val_loss: 0.2821 -

In [27]:
model.save_weights('Weights/LSTM.h5')

# Get embedding vectors

Running the following blocks produces two tsv files `vecs.tsv` and `meta.tsv`. These files can then be uploaded to [projector.tensorlow.org]() to view the embeddings in a 3d space

In [24]:
word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [25]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 64)


In [26]:
out_v = io.open('embeddings/vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('embeddings/meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

# Predict using custom string

Custom strings can then be tokenized by the `Tokenizer` class and the sentiment can be predicted using `model.predict()`

In [34]:
model.load_weights("Weights/LSTM.h5")

In [28]:
def predict_sentiment(model,custom_text):
    custom_sequence = tokenizer.texts_to_sequences(np.array([custom_text]))
    custom_padded = pad_sequences(custom_sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    if float(model.predict(custom_padded))>0.5:
        print("The review has a positive sentiment :)")
    else:
        print("The review has a negative sentiment :(")

In [35]:
test_string = "the food was really good and I loved it"

In [36]:
predict_sentiment(model,test_string)

The review has a positive sentiment :)
